# **New Zealand flood study - model analysis**

In [23]:
source("../methods_extRemes.r")


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# generic method to compute bootstrapped confidence intervals
boot_ci <- function(mdl, rp, cov1, cov2, ev = NA, nsamp = 10000, lower = F, rel = F) {
    
    n <- length(mdl$x)
    mtype <- strsplit(mdl$type, "_")[[1]]
            
    boot_sample <- rbind.fill(sapply(1:nsamp, function(i) {
        
        # resample & refit model
        boot_df <- mdl$cov.data[sample(1:n,n,replace = T),]
        suppressWarnings(boot_fit <- fevd_fixeddisp(mdl$var.name, mdl$cov.name, data = boot_df, method = "MLE", solver = solver))
        
        # compute required quantities for bootstrap (dispersion, PR, delta_I, delta_logI)
        pars_now <- sgev_pars(boot_fit, cov1)
        pars_then <- sgev_pars(boot_fit, cov2)
        disp <- pars_now$scale / pars_now$loc
        sigma <- pars_now$scale
        trend <- boot_fit$results$par["alpha"]
        shape <- boot_fit$results$par["shape"]
        
        mu0 <- pars_now$loc
        mu1 <- pars_then$loc
        
        if(is.na(ev)) {
            rl <- return_level(boot_fit, rp, cov1, lower = lower)
            rlhist <- return_level(boot_fit, rp, cov2, lower = lower)
        } else {
            rl <- ev
            rp <- return_period(boot_fit, ev, cov1, lower = lower)
            rlhist <- return_level(boot_fit, rp, cov2, lower = lower)
        }
        
        pr <- prob_ratio(boot_fit, rl, cov1, cov2, lower = lower)
        dI <- delta_I(boot_fit, rp, cov1, cov2, rel = F, lower = lower)
        dI_rel <- delta_I(boot_fit, rp, cov1, cov2, rel = T, lower = lower)
        
        return(as.data.frame(list("mu0" = mu0, "mu1" = mu1, "disp" = disp, "sigma" = sigma, "trend" = trend, "shape" = shape, "rp" = rp, "pr" = pr, "delta_I" = dI, "delta_I_rel" = dI)))
    }, simplify = F))
    return(apply(boot_sample, 2, quantile, c(0.025, 0.975)))
}

## **CORDEX**

In [24]:
rp <- 10
nboot = 10
lower = F
dI_rel = T
solver = "Nelder-Mead"

In [30]:
cordex_results <- rbind.fill(sapply(list.files("ts", pattern = "rx2day_AUS-22"), function(fnm) {
    
    set.seed(1)
    
    gmst_fnm <- list.files("ts", pattern = paste0("gsat_",strsplit(fnm, "_")[[1]][4]), full.names = T)

    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", "rx2day"))
    gmst <- read.csv(gmst_fnm, comment.char = "#",  header = F, sep = " ", col.names = c("year", "gmst"))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    event_gmst <- df["2023", "gmst"]
    
    mdl_eval <- fevd_fixeddisp("rx2day", "gmst", df[df$year >= 1979 & df$year <= 2023,], method = "MLE", solver = solver)  # obs period only
    mdl_attr <- fevd_fixeddisp("rx2day", "gmst", df[df$year <= 2023,], method = "MLE", solver = solver)                    # up to present day
    mdl_proj <- fevd_fixeddisp("rx2day", "gmst", df[df$year <= 2050,], method = "MLE", solver = solver)                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc                                # needs bootstrapping
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = lower)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = lower)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = lower)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = lower)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = lower)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = dI_rel, lower = lower)   
    attr_trend <- mdl_attr$results$par["alpha"]
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = lower)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = lower)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = lower)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, attr_rl, event_gmst, event_gmst + 0.8, lower = lower)                  # should switch cov1 & cov2 to get the right result, but sticking with inversion for consistency for now.
    proj_dI <- delta_I(mdl_proj, rp, event_gmst, event_gmst + 0.8, rel = dI_rel, lower = lower)
    proj_trend <- mdl_proj$results$par["alpha"]
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = nboot, lower = lower, rel = dI_rel)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst, cov2 = event_gmst + 0.8, ev = attr_rl, nsamp = nboot, lower = lower, rel = dI_rel)
    
    # return results (including inversion of the future values)
    cbind(model = gsub(".+AUS-22_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "eval_shape" = mdl_eval$results$par["shape"], "eval_shape_lower" = ci_eval["2.5%","shape"], "eval_shape_upper" = ci_eval["97.5%","shape"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_trend" = attr_trend, "attr_trend_lower" = ci_attr["2.5%","trend"], "attr_trend_upper" = ci_attr["97.5%","trend"],
                             "attr_value" = attr_rl, "attr_logvalue" = log10(attr_rl), "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "proj_trend" = proj_trend, "proj_trend_lower" = ci_proj["2.5%","trend"], "proj_trend_upper" = ci_proj["97.5%","trend"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_pr_inv" = 1/proj_pr, "proj_pr_lower_inv" = 1/ci_proj["97.5%","pr"], "proj_pr_upper_inv" = 1/ci_proj["2.5%","pr"],
                             "proj_dI_inv" = -proj_dI, "proj_dI_lower_inv" = -ci_proj["97.5%","delta_I"], "proj_dI_upper_inv" = -ci_proj["2.5%","delta_I"]
                             )))
    
}, simplify = F))

In [31]:
write.csv(cordex_results, "cordex_eval_results.csv", row.names = T)